In [143]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# 500 Cities Dataset
The 500 cities dataset is loaded into a Panda’s dataframe to reveal 810103 entries and 24 columns.

In [6]:
df_local = pd.read_csv("500_Cities__Local_Data_for_Better_Health__2018_release.csv")

In [7]:
#print(df_local.head())
print(df_local.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 810103 entries, 0 to 810102
Data columns (total 24 columns):
Year                          810103 non-null int64
StateAbbr                     810103 non-null object
StateDesc                     810103 non-null object
CityName                      810047 non-null object
GeographicLevel               810103 non-null object
DataSource                    810103 non-null object
Category                      810103 non-null object
UniqueID                      810103 non-null object
Measure                       810103 non-null object
Data_Value_Unit               810103 non-null object
DataValueTypeID               810103 non-null object
Data_Value_Type               810103 non-null object
Data_Value                    787311 non-null float64
Low_Confidence_Limit          787311 non-null float64
High_Confidence_Limit         787311 non-null float64
Data_Value_Footnote_Symbol    22794 non-null object
Data_Value_Footnote           22794 non-

In [8]:
#Filter for desired columns, remove redundant columns and rename columns
df_local = df_local[['Year', 'GeographicLevel', 'StateDesc', 'CityName','Short_Question_Text','Measure','Data_Value_Type', 
               'Data_Value','Low_Confidence_Limit', 'High_Confidence_Limit','PopulationCount','GeoLocation',
               'CityFIPS','TractFIPS']]
df_local.columns = ['Year', 'GeographicLevel','State','City','CategoryID','Measure','Data_Type', 
               'Data_Value','Low_Confidence_Limit', 'High_Confidence_Limit','Population','GeoLocation',
               'CityFIPS','TractFIPS']

In [9]:
df_local.columns

Index(['Year', 'GeographicLevel', 'State', 'City', 'CategoryID', 'Measure',
       'Data_Type', 'Data_Value', 'Low_Confidence_Limit',
       'High_Confidence_Limit', 'Population', 'GeoLocation', 'CityFIPS',
       'TractFIPS'],
      dtype='object')

In [10]:
df_local = df_local.loc[df_local['GeographicLevel']=='Census Tract']

In [144]:
df_local = df_local.dropna(subset=['Data_Value'])

We filter to show only rows related to our metric of interest. To do this, the ‘Measure’ column is filtered to contain only heart disease prevalence and the prevalences of risk factors for heart disease. These include hypertension, high cholesterol, smoking, diabetes, sedentarism and obesity.

In [12]:
disease = ['High blood pressure among adults aged >=18 Years','Coronary heart disease among adults aged >=18 Years',
           'High cholesterol among adults aged >=18 Years who have been screened in the past 5 Years',
           'Current smoking among adults aged >=18 Years','Diagnosed diabetes among adults aged >=18 Years',
            'No leisure-time physical activity among adults aged >=18 Years','Obesity among adults aged >=18 Years']

df_local = df_local[df_local['Measure'].isin(disease)]

In [13]:
smoking = df_local[df_local["CategoryID"] == "Current Smoking"]
cholesterol=df_local[df_local["CategoryID"] == "High Cholesterol"]
high_bp=df_local[df_local["CategoryID"] == "High Blood Pressure"]
heart=df_local[df_local["CategoryID"] == "Coronary Heart Disease"]
diabetes=df_local[df_local["CategoryID"] == "Diabetes"]
inactive=df_local[df_local["CategoryID"] == "Physical Inactivity"]
obesity=df_local[df_local["CategoryID"] == "Obesity"]


# HUD-USPS ZIP Crosswalk Files¶
The columns ‘zip’ (containing the zip code), ‘tract’ (containing the census tract) and ‘res_ratio’ (containing the percentage of census tract residents who reside in this zip code) are selected and stored.

In [14]:
# Read zip-to-tract conversion file
tractzip = pd.read_excel("Tract-zip.xlsx")
# Extract desired columns
tractzip = tractzip[["zip","tract","res_ratio"]]
tractzip.head()

,zip,tract,res_ratio
0,36067,1001020100,1.000000
1,36067,1001020200,1.000000
2,36067,1001020300,1.000000
3,36067,1001020400,0.025201
4,36066,1001020400,0.974799


The resulting dataframe is grouped by tract and the row with the maximum value of ‘res_ratio’ is selected for each tract. This is then merged back in with the original crosswalk df so that only one zip code, where most of the tract’s population resides, is represented for each tract.

In [145]:
tract_g=tractzip.groupby('tract')['res_ratio'].max().reset_index()
# Merge tract groups with the original df containing zip codes
# resulting in one zip code per tract
tractzipm = pd.merge(tractzip,tract_g, how="inner")
tractzipm.columns = ["Zip Code","tract","res_ratio"]
tractzipm = tractzipm[["Zip Code","tract"]]
display(tractzipm.head())
display(tractzipm.info())

,Zip Code,tract
0,36067,1001020100
1,36067,1001020200
2,36067,1001020300
3,36066,1001020400
4,36066,1001020500


<class 'pandas.core.frame.DataFrame'>
Int64Index: 73683 entries, 0 to 73682
Data columns (total 2 columns):
Zip Code    73683 non-null int64
tract       73683 non-null int64
dtypes: int64(2)
memory usage: 1.7 MB


None

# Merging 500 Cities and Zip Crosswalk
The crosswalk dataframe is merged with each of the health prevalences individually.

In [16]:
# Merge health values with conversion dataframe
smokingm = pd.merge(smoking,tractzipm,how="inner",left_on="TractFIPS",right_on="tract")
hypertensionm = pd.merge(high_bp,tractzipm,how="inner",left_on="TractFIPS",right_on="tract")
obesitym = pd.merge(obesity,tractzipm,how="inner",left_on="TractFIPS",right_on="tract")
inactivem = pd.merge(inactive,tractzipm,how="inner",left_on="TractFIPS",right_on="tract")
cholesterolm = pd.merge(cholesterol,tractzipm,how="inner",left_on="TractFIPS",right_on="tract")
diabetesm = pd.merge(diabetes,tractzipm,how="inner",left_on="TractFIPS",right_on="tract")
heartdm = pd.merge(heart,tractzipm,how="inner",left_on="TractFIPS",right_on="tract")

In [17]:
# Group by zip code for each value and calculate weighted mean prevalence
smokingmp = smokingm.groupby("Zip Code").apply(lambda x: np.average(x['Data_Value'].astype("float"),weights=x['Population'])).reset_index()
smokingmp.columns = ["Zip Code","Smoking Prevalence"]

hypertensionmp = hypertensionm.groupby("Zip Code").apply(lambda x: np.average(x['Data_Value'].astype("float"),weights=x['Population'])).reset_index()
hypertensionmp.columns = ["Zip Code","Hypertension Prevalence"]

obesitymp = obesitym.groupby("Zip Code").apply(lambda x: np.average(x['Data_Value'].astype("float"),weights=x['Population'])).reset_index()
obesitymp.columns = ["Zip Code","Obesity Prevalence"]

inactivemp = inactivem.groupby("Zip Code").apply(lambda x: np.average(x['Data_Value'].astype("float"),weights=x['Population'])).reset_index()
inactivemp.columns = ["Zip Code","Sedentarism Prevalence"]

cholesterolmp = cholesterolm.groupby("Zip Code").apply(lambda x: np.average(x['Data_Value'].astype("float"),weights=x['Population'])).reset_index()
cholesterolmp.columns = ["Zip Code","Cholesterol Prevalence"]

diabetesmp = diabetesm.groupby("Zip Code").apply(lambda x: np.average(x['Data_Value'].astype("float"),weights=x['Population'])).reset_index()
diabetesmp.columns = ["Zip Code","Diabetes Prevalence"]

heartmp = heartdm.groupby("Zip Code").apply(lambda x: np.average(x['Data_Value'].astype("float"),weights=x['Population'])).reset_index()
heartmp.columns = ["Zip Code","Heart Disease Prevalence"]

#Merge the values
_ = pd.merge(smokingmp,hypertensionmp, on="Zip Code")
_ = pd.merge(_,obesitymp, on="Zip Code")
_ = pd.merge(_,inactivemp, on="Zip Code")
_ = pd.merge(_,cholesterolmp, on="Zip Code")
_ = pd.merge(_,diabetesmp, on="Zip Code")
datameans = pd.merge(_,heartmp, on="Zip Code")

display(datameans.head())
display(datameans.info())

,Zip Code,Smoking Prevalence,Hypertension Prevalence,Obesity Prevalence,Sedentarism Prevalence,Cholesterol Prevalence,Diabetes Prevalence,Heart Disease Prevalence
0,1101,28.300000,36.400000,44.000000,46.600000,41.300000,19.300000,9.300000
1,1104,24.269272,33.257045,37.437441,37.721353,38.392990,14.837576,8.816472
2,1105,28.596151,32.879892,42.176132,43.694299,37.409454,15.840128,7.849705
3,1107,25.495498,31.912761,41.052003,44.608772,38.631033,16.712049,8.234889
4,1108,23.351695,29.923640,34.742488,33.123923,35.242800,12.185663,6.873862


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4796 entries, 0 to 4795
Data columns (total 8 columns):
Zip Code                    4796 non-null int64
Smoking Prevalence          4796 non-null float64
Hypertension Prevalence     4796 non-null float64
Obesity Prevalence          4796 non-null float64
Sedentarism Prevalence      4796 non-null float64
Cholesterol Prevalence      4796 non-null float64
Diabetes Prevalence         4796 non-null float64
Heart Disease Prevalence    4796 non-null float64
dtypes: float64(7), int64(1)
memory usage: 337.2 KB


None

# Fastfoodmaps.com
This dataset contains a row for each fast food restaurant and 8 unnamed columns containing restaurant name, address, city, state, zip code, phone number, longitude, latitude.

In [18]:
#Open and import the fast food csv into a dataframe
df_food = pd.read_csv("fastfoodmaps_locations_2007.csv", header=None)
df_food.head()

,0,1,2,3,4,5,6,7,8,9
0,1,b,3601 N.W. 27th Avenue,Miami,FL,33142,(305) 638-3838,25.8092,-80.2400,0
1,2,b,8995 N. W. 7th Avenue,Miami,FL,33150,(305) 754-8453,25.8587,-80.2094,0
2,3,b,30390 South Dixie Highway,Homestead,FL,33030,(305) 247-7181,25.4849,-80.4610,0
3,4,b,7975 N. W. 27th Avenue,Miami,FL,33147,(305) 836-8152,25.8471,-80.2415,0
4,5,b,9201 South Dixie Highway,Miami,FL,33156,(305) 666-1130,25.6849,-80.3125,0


In [19]:
df_food = df_food[[1,3,4,5,7,8]]
df_food.columns =["Restaurant Count","City","State","Zip Code","Latitude","Longitude"]
df_food['Restaurant Count'] = 1


In [22]:
df_food.head()
df_food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50002 entries, 0 to 50001
Data columns (total 6 columns):
Restaurant Count    50002 non-null int64
City                50002 non-null object
State               50002 non-null object
Zip Code            50002 non-null object
Latitude            50002 non-null float64
Longitude           50002 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 2.3+ MB


The zip code column contains entries with trailing digits beyond the 5th digit. These trailing digits are removed.

In [36]:
nohyphen = []
for x in df_food["Zip Code"]:
    nohyphen.append(x.replace('-',''))
df_food["Zip Code"] = nohyphen

zlist = []

for x in df_food["Zip Code"]:
    if len(x) > 5:
        zlist.append(x[:5])
    else:
        zlist.append(x)
        
df_food["Zip Code"] = zlist

#f_food["Zip Code"].sort_values()
df_food.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50002 entries, 0 to 50001
Data columns (total 6 columns):
Restaurant Count    50002 non-null int64
City                50002 non-null object
State               50002 non-null object
Zip Code            50002 non-null object
Latitude            50002 non-null float64
Longitude           50002 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 2.3+ MB


    The dataframe is grouped by zip code and a sum of restaurants is calculated.

In [56]:
fgroup = df_food.groupby("Zip Code")["Restaurant Count"].sum().reset_index()
fgroup["Zip Code"] = fgroup["Zip Code"].astype('int')
fgroup.head()
fgroup.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11636 entries, 0 to 11635
Data columns (total 2 columns):
Zip Code            11636 non-null int32
Restaurant Count    11636 non-null int64
dtypes: int32(1), int64(1)
memory usage: 136.4 KB


Finally, this dataframe is merged with the working health dataframe, and zeroes are filled in for zip codes that are missing restaurant counts. Here we assumed that these zip codes have no fast food restaurants.

In [59]:

datameansm = pd.merge(datameans,fgroup,how="left",on="Zip Code")

# Fill NaN values with 0
datameansm = datameansm.fillna(0)

display(datameansm.head())
display(datameansm.info())
display(datameansm.describe())

,Zip Code,Smoking Prevalence,Hypertension Prevalence,Obesity Prevalence,Sedentarism Prevalence,Cholesterol Prevalence,Diabetes Prevalence,Heart Disease Prevalence,Restaurant Count
0,1101,28.300000,36.400000,44.000000,46.600000,41.300000,19.300000,9.300000,2.0
1,1104,24.269272,33.257045,37.437441,37.721353,38.392990,14.837576,8.816472,5.0
2,1105,28.596151,32.879892,42.176132,43.694299,37.409454,15.840128,7.849705,4.0
3,1107,25.495498,31.912761,41.052003,44.608772,38.631033,16.712049,8.234889,1.0
4,1108,23.351695,29.923640,34.742488,33.123923,35.242800,12.185663,6.873862,2.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4796 entries, 0 to 4795
Data columns (total 9 columns):
Zip Code                    4796 non-null int64
Smoking Prevalence          4796 non-null float64
Hypertension Prevalence     4796 non-null float64
Obesity Prevalence          4796 non-null float64
Sedentarism Prevalence      4796 non-null float64
Cholesterol Prevalence      4796 non-null float64
Diabetes Prevalence         4796 non-null float64
Heart Disease Prevalence    4796 non-null float64
Restaurant Count            4796 non-null float64
dtypes: float64(8), int64(1)
memory usage: 374.7 KB


None

,Zip Code,Smoking Prevalence,Hypertension Prevalence,Obesity Prevalence,Sedentarism Prevalence,Cholesterol Prevalence,Diabetes Prevalence,Heart Disease Prevalence,Restaurant Count
count,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000
mean,59471.652836,18.076896,30.129029,30.072722,24.871650,34.342210,10.458133,5.748891,4.273144
std,29108.723921,5.562125,7.728308,7.644389,8.593688,5.085662,3.932428,1.928120,4.142664
min,1101.000000,2.470610,5.700000,10.519778,8.300000,8.500000,0.800000,0.500000,0.000000
25%,33426.750000,14.030588,25.322792,24.532705,18.161810,31.856236,7.766324,4.513961,1.000000
50%,66104.500000,17.444661,29.181122,29.300601,23.573884,34.775364,9.777981,5.629315,3.000000
75%,85299.500000,21.577710,34.002288,34.819921,30.652700,37.499350,12.485496,6.882875,7.000000
max,99577.000000,40.900000,69.300000,56.000000,52.308167,52.100000,28.158736,21.000000,24.000000


# Uszipcode

This library is imported and a search engine to retrieve all of its rows (one for each zip code) is created and run

In [62]:
# Import uszipcode to retrieve income for zip codes
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)
zipstats = search.by_median_household_income(lower=-1, upper=2147483648, zipcode_type='Standard', 
                                            sort_by='median_household_income', ascending=False, returns=999999)

The resulting list is transformed to a dictionary and used to create a dataframe.

In [64]:
# Transform list of dictionaries to dataframe
list = []
for entry in zipstats:
    list.append(entry.to_dict())

zipstats_df = pd.DataFrame(list)
display(zipstats_df.head())

,area_code_list,bounds_east,bounds_north,bounds_south,bounds_west,common_city_list,county,housing_units,land_area_in_sqmi,lat,...,occupied_housing_units,population,population_density,post_office_city,radius_in_miles,state,timezone,water_area_in_sqmi,zipcode,zipcode_type
0,[970],-108.683032,37.573355,37.435026,-108.849361,[Yellow Jacket],Montezuma County,64,27.12,37.540,...,54,131,5.0,"Yellow Jacket, CO",2.000000,CO,Mountain,0.00,81335,Standard
1,"[301, 410, 443]",-76.529878,39.040506,39.017669,-76.570564,"[Annapolis, Sherwood Forest, Sherwood Frst]",Anne Arundel County,340,0.99,39.030,...,203,544,549.0,"Annapolis, MD",0.965909,MD,Eastern,0.75,21405,Standard
2,[503],-121.679473,45.366297,45.215218,-121.872530,"[Government Camp, Government Cp, Timberline Lo...",Clackamas County,713,38.01,45.300,...,114,217,6.0,"Government Camp, OR",2.000000,OR,Pacific,0.10,97028,Standard
3,"[281, 346, 713, 832]",-95.355302,29.757155,29.749765,-95.364051,[Houston],Harris County,471,0.12,29.754,...,249,366,2965.0,"Houston, TX",0.284091,TX,Central,0.00,77010,Standard
4,[307],-105.793154,41.128117,40.514119,-106.195438,"[Jelm, Laramie]",Albany County,170,384.84,41.060,...,54,100,0.0,"Jelm, WY",6.000000,WY,Mountain,0.85,82063,Standard


The ‘zip code’, ‘median_household_income’, and ‘population’ columns are selected and stored. The zip code column is changed from object to integer so it may be merged.

In [67]:
# Select zipcode and income columns and rename columns
zipincome = zipstats_df[["zipcode","median_household_income","population"]]
zipincome.columns = ["Zip Code","median_household_income","Population"]

# Convert zip codes to integers
zipincome["Zip Code"] = zipincome["Zip Code"].astype("int")

display(zipincome.head())
display(zipincome.info())

,Zip Code,median_household_income,Population
0,81335,250001,131
1,21405,250001,544
2,97028,250001,217
3,77010,250001,366
4,82063,250001,100


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28847 entries, 0 to 28846
Data columns (total 3 columns):
Zip Code                   28847 non-null int32
median_household_income    28847 non-null int64
Population                 28847 non-null int64
dtypes: int32(1), int64(2)
memory usage: 563.5 KB


None

This dataframe is merged with the working health dataframe. Missing population data is dropped.

In [69]:
# Merge income data with dataframe
_ = pd.merge(datameansm,zipincome,how="left",on="Zip Code")

display(_.head())
display(_.info())
data = _.copy()

,Zip Code,Smoking Prevalence,Hypertension Prevalence,Obesity Prevalence,Sedentarism Prevalence,Cholesterol Prevalence,Diabetes Prevalence,Heart Disease Prevalence,Restaurant Count,median_household_income,Population
0,1101,28.300000,36.400000,44.000000,46.600000,41.300000,19.300000,9.300000,2.0,NaN,NaN
1,1104,24.269272,33.257045,37.437441,37.721353,38.392990,14.837576,8.816472,5.0,32273.0,22865.0
2,1105,28.596151,32.879892,42.176132,43.694299,37.409454,15.840128,7.849705,4.0,18402.0,12350.0
3,1107,25.495498,31.912761,41.052003,44.608772,38.631033,16.712049,8.234889,1.0,21737.0,11611.0
4,1108,23.351695,29.923640,34.742488,33.123923,35.242800,12.185663,6.873862,2.0,34064.0,26688.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4796 entries, 0 to 4795
Data columns (total 11 columns):
Zip Code                    4796 non-null int64
Smoking Prevalence          4796 non-null float64
Hypertension Prevalence     4796 non-null float64
Obesity Prevalence          4796 non-null float64
Sedentarism Prevalence      4796 non-null float64
Cholesterol Prevalence      4796 non-null float64
Diabetes Prevalence         4796 non-null float64
Heart Disease Prevalence    4796 non-null float64
Restaurant Count            4796 non-null float64
median_household_income     4400 non-null float64
Population                  4400 non-null float64
dtypes: float64(10), int64(1)
memory usage: 449.6 KB


None

# Outliers

The heart prevalence column ranges from 0.5 to 21 with a mean of 5.7. While some statistical outliers may exist in the upper ranges, I feel these are valuable observations for the analysis and should not be removed. Few outliers are present in the prevalences of the risk factors, which are also valuable for analysis.

In [71]:
display(data.describe())

,Zip Code,Smoking Prevalence,Hypertension Prevalence,Obesity Prevalence,Sedentarism Prevalence,Cholesterol Prevalence,Diabetes Prevalence,Heart Disease Prevalence,Restaurant Count,median_household_income,Population
count,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4796.000000,4400.000000,4400.000000
mean,59471.652836,18.076896,30.129029,30.072722,24.871650,34.342210,10.458133,5.748891,4.273144,55590.915909,29040.904091
std,29108.723921,5.562125,7.728308,7.644389,8.593688,5.085662,3.932428,1.928120,4.142664,24464.347788,17435.719895
min,1101.000000,2.470610,5.700000,10.519778,8.300000,8.500000,0.800000,0.500000,0.000000,6067.000000,219.000000
25%,33426.750000,14.030588,25.322792,24.532705,18.161810,31.856236,7.766324,4.513961,1.000000,38264.750000,16248.000000
50%,66104.500000,17.444661,29.181122,29.300601,23.573884,34.775364,9.777981,5.629315,3.000000,51155.500000,26711.500000
75%,85299.500000,21.577710,34.002288,34.819921,30.652700,37.499350,12.485496,6.882875,7.000000,68428.000000,38711.750000
max,99577.000000,40.900000,69.300000,56.000000,52.308167,52.100000,28.158736,21.000000,24.000000,230952.000000,113916.000000


# Grouping features

Each zip code will be grouped into a population category and a restaurant number category. This will be useful for exploring the data and statistical analysis.

In [74]:
# Select dataframe without missing values
data_notna = data.dropna()

# Create a new column specifying population group by size
popgroups = []
for pop in data_notna["Population"]:
    if pop >80000:
        popgroups.append(">80000")
    if pop <20000:
        popgroups.append("<20000")
    if pop in range(20000,40000):
        popgroups.append("20000-40000")
    if pop in range(40000,60000):
        popgroups.append("40000-60000")
    if pop in range(60000,80000):
        popgroups.append("60000-80000")
        
data_notna["Population Group"] = popgroups
data_notna.head()

,Zip Code,Smoking Prevalence,Hypertension Prevalence,Obesity Prevalence,Sedentarism Prevalence,Cholesterol Prevalence,Diabetes Prevalence,Heart Disease Prevalence,Restaurant Count,median_household_income,Population,Population Group
1,1104,24.269272,33.257045,37.437441,37.721353,38.392990,14.837576,8.816472,5.0,32273.0,22865.0,20000-40000
2,1105,28.596151,32.879892,42.176132,43.694299,37.409454,15.840128,7.849705,4.0,18402.0,12350.0,<20000
3,1107,25.495498,31.912761,41.052003,44.608772,38.631033,16.712049,8.234889,1.0,21737.0,11611.0,<20000
4,1108,23.351695,29.923640,34.742488,33.123923,35.242800,12.185663,6.873862,2.0,34064.0,26688.0,20000-40000
5,1109,23.329629,32.285360,37.212078,33.779183,34.154532,13.474276,6.706302,2.0,33376.0,30250.0,20000-40000


In [112]:
data_notna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4400 entries, 1 to 4795
Data columns (total 14 columns):
Zip Code                    4400 non-null int64
Smoking Prevalence          4400 non-null float64
Hypertension Prevalence     4400 non-null float64
Obesity Prevalence          4400 non-null float64
Sedentarism Prevalence      4400 non-null float64
Cholesterol Prevalence      4400 non-null float64
Diabetes Prevalence         4400 non-null float64
Heart Disease Prevalence    4400 non-null float64
Restaurant Count            4400 non-null float64
median_household_income     4400 non-null float64
Population                  4400 non-null float64
Population Group            4400 non-null object
Restaurant Group            4400 non-null object
Restaurant Gp               4034 non-null object
dtypes: float64(10), int64(1), object(3)
memory usage: 515.6+ KB


In [139]:
# Create a new column specifying restaurant group by sum
restgroups = []
for count in data_notna["Restaurant Count"]:
    if count in range(0,8):
        restgroups.append("0-7")
    if count in range(8, 25):
        restgroups.append("8-24")
        
data_notna["Restaurant Group"]= restgroups
data_notna


,Zip Code,Smoking Prevalence,Hypertension Prevalence,Obesity Prevalence,Sedentarism Prevalence,Cholesterol Prevalence,Diabetes Prevalence,Heart Disease Prevalence,Restaurant Count,median_household_income,Population,Population Group,Restaurant Group
1,1104,24.269272,33.257045,37.437441,37.721353,38.392990,14.837576,8.816472,5.0,32273.0,22865.0,20000-40000,0-7
2,1105,28.596151,32.879892,42.176132,43.694299,37.409454,15.840128,7.849705,4.0,18402.0,12350.0,<20000,0-7
3,1107,25.495498,31.912761,41.052003,44.608772,38.631033,16.712049,8.234889,1.0,21737.0,11611.0,<20000,0-7
4,1108,23.351695,29.923640,34.742488,33.123923,35.242800,12.185663,6.873862,2.0,34064.0,26688.0,20000-40000,0-7
5,1109,23.329629,32.285360,37.212078,33.779183,34.154532,13.474276,6.706302,2.0,33376.0,30250.0,20000-40000,0-7
6,1118,19.461619,30.633531,30.483470,26.105537,35.860784,10.599838,7.075118,1.0,62283.0,14071.0,<20000,0-7
7,1119,21.656562,30.136570,33.192432,29.277953,34.842082,11.433115,6.894488,2.0,46055.0,14152.0,<20000,0-7
8,1128,18.200000,29.600000,28.900000,22.900000,35.400000,9.800000,6.600000,0.0,78864.0,2631.0,<20000,0-7
9,1129,19.300000,31.600000,31.300000,26.100000,36.200000,11.000000,6.800000,2.0,61435.0,7019.0,<20000,0-7
11,1151,26.627879,31.053690,37.070909,35.386489,35.904100,12.461070,7.454510,0.0,30043.0,8698.0,<20000,0-7


Missing values are returned so that imputation can be performed later.

In [140]:
data = pd.merge(data, data_notna, how="left")

In [142]:
# Export to file
export_csv = data.to_csv('heart_disease_restaurant.csv', header=True)